In [55]:
# Import dependencies
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from flask import Flask, render_template
import pymongo
from flask_pymongo import PyMongo

In [56]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [57]:
# read & scrape the information from table via panda
url = 'https://www.timeanddate.com/weather/?low=c&sort=1'
tables = pd.read_html(url)
tables

[    Local Time and Weather in Capitals Around the World Sort By: CityCountryTimeTemperatureCities Shown: Capitals (215)Most Popular (143)Popular (356)Somewhat Popular (469)  \
 0                                      Albania, Tirana                                                                                                                         
 1                                     Algeria, Algiers                                                                                                                         
 2                                       Angola, Luanda                                                                                                                         
 3                    Antigua and Barbuda, Saint John's                                                                                                                         
 4                              Argentina, Buenos Aires                                                            

In [59]:
type(tables)

list

In [72]:
# Make it into tables
df = tables[0]
df.columns = ['Country', 'Time', 'City', 'Temperature']
df

,Country,Time,City,Temperature
0,"Albania, Tirana",Mon 1:33 pm,NaN,13 °C
1,"Algeria, Algiers",Mon 1:33 pm,NaN,12 °C
2,"Angola, Luanda",Mon 1:33 pm,NaN,29 °C
3,"Antigua and Barbuda, Saint John's",Mon 8:33 am,NaN,25 °C
4,"Argentina, Buenos Aires",Mon 9:33 am,NaN,20 °C
...,...,...,...,...
184,"Vatican City State, Vatican City",Mon 1:33 pm,NaN,7 °C
185,"Venezuela, Caracas",Mon 8:33 am,NaN,26 °C
186,"Vietnam, Hanoi",Mon 7:33 pm,NaN,14 °C
187,"Zambia, Lusaka",Mon 2:33 pm,NaN,NaN


In [73]:
# drop Columes and nan value
df = df.drop(columns=['City'])
df

,Country,Time,Temperature
0,"Albania, Tirana",Mon 1:33 pm,13 °C
1,"Algeria, Algiers",Mon 1:33 pm,12 °C
2,"Angola, Luanda",Mon 1:33 pm,29 °C
3,"Antigua and Barbuda, Saint John's",Mon 8:33 am,25 °C
4,"Argentina, Buenos Aires",Mon 9:33 am,20 °C
...,...,...,...
184,"Vatican City State, Vatican City",Mon 1:33 pm,7 °C
185,"Venezuela, Caracas",Mon 8:33 am,26 °C
186,"Vietnam, Hanoi",Mon 7:33 pm,14 °C
187,"Zambia, Lusaka",Mon 2:33 pm,NaN


In [74]:
df = df.dropna()
df

,Country,Time,Temperature
0,"Albania, Tirana",Mon 1:33 pm,13 °C
1,"Algeria, Algiers",Mon 1:33 pm,12 °C
2,"Angola, Luanda",Mon 1:33 pm,29 °C
3,"Antigua and Barbuda, Saint John's",Mon 8:33 am,25 °C
4,"Argentina, Buenos Aires",Mon 9:33 am,20 °C
...,...,...,...
182,"Uzbekistan, Tashkent",Mon 5:33 pm,8 °C
183,"Vanuatu, Port Vila",Mon 11:33 pm,25 °C
184,"Vatican City State, Vatican City",Mon 1:33 pm,7 °C
185,"Venezuela, Caracas",Mon 8:33 am,26 °C


In [76]:
# split country into country and city
df[['Country', 'B']] = df['Country'].str.split(', ', 1, expand=True)
df

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Country,Time,Temperature,B
0,Albania,Mon 1:33 pm,13 °C,Tirana
1,Algeria,Mon 1:33 pm,12 °C,Algiers
2,Angola,Mon 1:33 pm,29 °C,Luanda
3,Antigua and Barbuda,Mon 8:33 am,25 °C,Saint John's
4,Argentina,Mon 9:33 am,20 °C,Buenos Aires
...,...,...,...,...
182,Uzbekistan,Mon 5:33 pm,8 °C,Tashkent
183,Vanuatu,Mon 11:33 pm,25 °C,Port Vila
184,Vatican City State,Mon 1:33 pm,7 °C,Vatican City
185,Venezuela,Mon 8:33 am,26 °C,Caracas


In [77]:
# drop column Time and B
df = df.drop(columns=['Time', 'B'])
df

,Country,Temperature
0,Albania,13 °C
1,Algeria,12 °C
2,Angola,29 °C
3,Antigua and Barbuda,25 °C
4,Argentina,20 °C
...,...,...
182,Uzbekistan,8 °C
183,Vanuatu,25 °C
184,Vatican City State,7 °C
185,Venezuela,26 °C


In [85]:
# export to csv
df.to_csv('output2.csv', index=False)

In [86]:
import os


In [87]:
# generate path to csv
csv_path = os.path.join('output2.csv')
temp_csv = pd.read_csv(csv_path)

In [88]:
# create html table and save to file 
temp_csv.to_html('table.html', index=False, classes=['table', 'table-striped', 'table-hover'])